 # Populacja bazy sql
 Zbieram dane z plików csv zapisanych w 1 notatniku i wgrywam do baz danych sql


Importuję biblioteki

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

import os
from time import time

Konfiguruję połączenie z bazą danych

In [2]:
# Poniżej
user = 'XYZ'
password = 'XYZ'
host = '127.0.0.1'
db_name = 'Airlines'

url = f'postgresql://{user}:{password}@{host}/{db_name}'
engine = create_engine(url)

 Ładuję ramki do obszaru roboczego i zmieniam nazwy kolumn na lower case

In [4]:
def load_raw_data(file_name):
    raw_data = pd.read_csv(file_name, sep=';')
    modified_columns = [column.lower() for column in raw_data.columns]
    raw_data.columns = modified_columns
    return raw_data

In [5]:
table_names = ['aircraft', 'airport_weather', 'flight', 'airport_list']

In [6]:
# Tworzę scieżki do plików

CURRENT_DIR = os.getcwd()
PARENT_DIR = os.path.abspath(os.path.join(CURRENT_DIR, os.pardir))
file_paths = {}
for table_name in table_names:
    file_path = f'{PARENT_DIR}\\data\\raw\\{table_name}.csv'
    file_paths[table_name] = file_path

In [7]:
#sprawdzam powstałe ścieżki
file_paths

{'aircraft': 'C:\\Users\\miejs\\Desktop\\CodersLab Exam\\data\\raw\\aircraft.csv',
 'airport_weather': 'C:\\Users\\miejs\\Desktop\\CodersLab Exam\\data\\raw\\airport_weather.csv',
 'flight': 'C:\\Users\\miejs\\Desktop\\CodersLab Exam\\data\\raw\\flight.csv',
 'airport_list': 'C:\\Users\\miejs\\Desktop\\CodersLab Exam\\data\\raw\\airport_list.csv'}

In [8]:
raw_data = {}
for table_name, file_path in file_paths.items():
    raw_data[table_name] = load_raw_data(file_path)

Eksport danych pandas na bazę sql


In [9]:
def export_table_to_db(df, table_name, engine):
    df.to_sql(name=table_name, con=engine, chunksize=100, if_exists='replace', index=False)

Wgrywam dane pandas do poszczególnych tabel

In [10]:
# Wgrywam wszystkie tabele od razu

for table_name, raw_data_df in raw_data.items():
    start = time()
    print(f'Uploading {raw_data_df.shape[0]} rows to table {table_name}.')
    export_table_to_db(raw_data_df, table_name, engine)
    end = time()
    print(f'Uploaded {raw_data_df.shape[0]} rows to table {table_name} took {end - start}s.')

Uploading 7383 rows to table aircraft.
Uploaded 7383 rows to table aircraft took 0.15983963012695312s.
Uploading 46226 rows to table airport_weather.
Uploaded 46226 rows to table airport_weather took 2.498711109161377s.
Uploading 9251880 rows to table flight.
Uploaded 9251880 rows to table flight took 500.53441166877747s.
Uploading 97 rows to table airport_list.
Uploaded 97 rows to table airport_list took 0.16103434562683105s.
